In [1]:
# Data processing
import pandas as pd

test_prompt_data = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/test_prompt_data_task1_simple.csv")
print(test_prompt_data.shape)
test_prompt_data.head()

(2312, 3)


,cell_id,prompt,answer
0,ACH-000006,Think step by step and decide in a single word...,Sensitive
1,ACH-000006,Think step by step and decide in a single word...,Resistant
2,ACH-000006,Think step by step and decide in a single word...,Resistant
3,ACH-000006,Think step by step and decide in a single word...,Sensitive
4,ACH-000006,Think step by step and decide in a single word...,Sensitive


In [2]:
test_prompt_data.iloc[219:,:] # N= 219

,cell_id,prompt,answer
219,ACH-000139,Think step by step and decide in a single word...,Sensitive
220,ACH-000139,Think step by step and decide in a single word...,Sensitive
221,ACH-000139,Think step by step and decide in a single word...,Resistant
222,ACH-000139,Think step by step and decide in a single word...,Resistant
223,ACH-000139,Think step by step and decide in a single word...,Resistant
...,...,...,...
2307,ACH-001654,Think step by step and decide in a single word...,Sensitive
2308,ACH-001654,Think step by step and decide in a single word...,Sensitive
2309,ACH-001654,Think step by step and decide in a single word...,Sensitive
2310,ACH-001654,Think step by step and decide in a single word...,Resistant


In [3]:
test_samples = test_prompt_data.iloc[:200,:].copy()
test_samples['output70'] = [None,]*200
# test_samples = pd.read_csv('./data/sensitivity/test_fewshot_simple.csv')

In [4]:
test_samples.head()

,cell_id,prompt,answer,output70
0,ACH-000006,Think step by step and decide in a single word...,Sensitive,None
1,ACH-000006,Think step by step and decide in a single word...,Resistant,None
2,ACH-000006,Think step by step and decide in a single word...,Resistant,None
3,ACH-000006,Think step by step and decide in a single word...,Sensitive,None
4,ACH-000006,Think step by step and decide in a single word...,Sensitive,None


In [5]:
fewshot_set = test_prompt_data.iloc[219:,:] #1. not appear in train data; 2. not to inference (test data top 200 -> inf)
fewshot_set.shape

(2093, 3)

In [6]:
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
model_dir = "/data/yingfei/models/llm/medAlpaca/medalpaca-7b"

model = LlamaForCausalLM.from_pretrained(model_dir, device_map = 'auto')#.to(DEVICE)
tokenizer = LlamaTokenizer.from_pretrained(model_dir)#.to(model.device)

# from inferer import Inferer
# medalpaca = Inferer(
#     model_name="medalpaca/medalapca-7b", 
#     prompt_template="path/to/your/prompt_template.json",
#     base_model=None,  # Optional, specify if using LoRA or another base model
#     model_max_length=512,  # Optional, can be adjusted based on your input needs
#     load_in_8bit=False,  # Optional, set to True if using a quantized model
#     torch_dtype=torch.float16,  # Optional, can be adjusted based on your precision needs
#     peft=False  # Optional, set to True if the model was trained with PEFT or in 8bit
# )

/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
import numpy as np

def get_few_shot(fewshot_set, num=5):
    fewshot = ''
    for i in range(num):#len(fewshot_set)):
        idx = np.random.randint(0,len(fewshot_set))
        answer = fewshot_set.iloc[idx].answer
        question = fewshot_set.iloc[idx].prompt.split("[Reasoning].")[1].replace('?',answer)#.replace("\n", "")
        fewshot += f'\n\nExample {i}:\n'
        fewshot += question#.strip()
    
    return fewshot.strip()

In [8]:
for idx in range(200):
    row = test_samples.iloc[idx,:]
    if not pd.isna(row.output70): continue
    print(f'PROCESSING CELLINE: {row.cell_id}   ', idx)
    prompt = row.prompt

    pipeline = transformers.pipeline("text-generation", model=model,
                                    tokenizer=tokenizer, temperature = 0.0,
                                    torch_dtype=torch.float16, device_map="auto",
                                    return_full_text=False)
    instruction = "Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]"
    prompt = prompt.split("[Reasoning].")[1]
    fewshot = get_few_shot(fewshot_set, num=5)
    prompt_content = f"<s>[INST] <<SYS>>{instruction}<</SYS>> <<EXAMPLE>>{fewshot}<</EXAMPLE>> {prompt}[/INST]"
    
    print(prompt_content)
    # Run prompt and pipeline
    sequences = pipeline(prompt_content, truncation=True, max_length=20480)
    # print(result[0]['generated_text']) # Not in one word??? !!!
    output = ""
    for seq in sequences:
        new_output = seq['generated_text']
        print(new_output)
        output += new_output
    

    test_samples.loc[idx,'output70'] = output.strip()
    test_samples.to_csv('test_output.csv', index=None)
    print()
    print(f'GROUND TRUTH: {row.answer}')
    print('*'*20)

PROCESSING CELLINE: ACH-000006    0
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> <<EXAMPLE>>Example 0:

Drug and cell line mutations: 
The drug is KIN001-135. The drug SMILES structure is COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=CC=C4S(=O)(=O)C)OC. Drug target is IKK. Drug target pathway is Other, kinases.
The mutations of the cell line are CDKN1A, DAXX, FAM46C, HGF, KDM6A, KEL, NOTCH1, NOTCH3, RB1, ROS1, TP53.
Drug Sensitivity: Sensitive

Example 1:

Drug and cell line mutations: 
The drug is PD-0325901. The drug SMILES structure is C1=CC(=C(C=C1I)F)NC2=C(C=CC(=C2F)F)C(=O)NOCC(CO)O. Drug target is MEK1, MEK2. Drug target pathway is ERK MAPK signaling.
The mutations of the cell line are CREBBP, EP300, KRAS, PBRM1, RET, RNF43, TGFBR2.
Drug Sensitivity: Resistant

Example 2:

Drug and cell line mutations: 
The drug is GSK2606414. The drug SMILES structure is CN1C=C(C2=C(N=C

/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<</SYS>>

GROUND TRUTH: Sensitive
********************
PROCESSING CELLINE: ACH-000006    1
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> <<EXAMPLE>>Example 0:

Drug and cell line mutations: 
The drug is YM-201636. The drug SMILES structure is C1COCCN1C2=NC(=NC3=C2OC4=C3C=CC=N4)C5=CC(=CC=C5)NC(=O)C6=CN=C(C=C6)N. Drug target is PIKFYVE. Drug target pathway is PI3K/MTOR signaling.
The mutations of the cell line are ATRX, CBL, CDKN2A, MET, ROS1, TP53.
Drug Sensitivity: Sensitive

Example 1:

Drug and cell line mutations: 
The drug is Z-LLNLE-CHO. The drug SMILES structure is CCCCC(C=O)NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](CC(C)C)NC(=O)OCC1=CC=CC=C1. Drug target is gamma-secretase. Drug target pathway is Other.
The mutations of the cell line are APC, BCR, BRCA1, ERBB2, ERBB4, FGFR3, FLT1, GNAS, IKZF1, KDM6A, MED12, MTOR, NOTCH1, PIK3CA, RB1, RET, SPEN, TP53.
Drug Sensitivity: Sensitive

E

/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> <<EXAMPLE>>Example 4:

Drug and cell line mutations: 
The drug is XMD8-92. The drug SMILES structure is CCOC1=C(C=CC(=C1)N2CCC(CC2)O)NC3=NC=C4C(=N3)N(C5=CC=CC=C5C(=O)N4C)C. Drug target is ERK5. Drug target pathway is ERK MAPK signaling.
The mutations of the cell line are ARID1A, BCOR, BRAF, ETV6, GATA3, GNAS, HGF, IDH1, IKZF1, MLL2, MED12, NKX2-1, NOTCH1, PIK3CA, POLD1, PTEN, RBM10, SMARCA4.
Drug Sensitivity: Resistant

Example 5:

Drug and cell line mutations: 
The drug is XMD14-99. The drug SMILES structure is CN1CCN(CC1)C2=CC(=C(C=C2)NC3=NC(=C(S3)C(=O)C4=C(C=CC=C4Cl)Cl)N)OC. Drug target is ALK, CDK7, LTK, others. Drug target pathway is Other, kinases.
The mutations of the cell line are APC, CARD11, EPHA3, FANCG, GABRA6, GATA4, KDM6A, NOTCH3, TP53.
Drug Sensitivity: Resistant

Example 6:

Drug and cell line mutations: 
The drug is QL-X-138

...



GROUND TRUTH: Sensitive
********************
PROCESSING CELLINE: ACH-000006    5
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> <<EXAMPLE>>Example 0:

Drug and cell line mutations: 
The drug is BELINOSTAT. The drug SMILES structure is C1=CC=C(C=C1)NS(=O)(=O)C2=CC=CC(=C2)C=CC(=O)NO. Drug target is HDAC1. Drug target pathway is Chromatin histone acetylation.
The mutations of the cell line are ATM, DAXX, GNAS, JAK2, KRAS.
Drug Sensitivity: Resistant

Example 1:

Drug and cell line mutations: 
The drug is APITOLISIB. The drug SMILES structure is CC1=C(SC2=C1N=C(N=C2N3CCOCC3)C4=CN=C(N=C4)N)CN5CCN(CC5)C(=O)[C@H](C)O. Drug target is mTOR, PI3K. Drug target pathway is PI3K/MTOR signaling.
The mutations of the cell line are FAM123B, APC, ATM, FBXW7, KRAS, MYC, TP53.
Drug Sensitivity: Resistant

Example 2:

Drug and cell line mutations: 
The drug is ZG-10. The drug SMILES structure is O

/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(




GROUND TRUTH: Resistant
********************
PROCESSING CELLINE: ACH-000006    6
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> <<EXAMPLE>>Example 0:

Drug and cell line mutations: 
The drug is KRAS (G12C) INHIBITOR-12. The drug SMILES structure is C=CC(=O)N1CCN(CC1)C(=O)CNC2=CC(=C(C=C2O)Cl)I. Drug target is KRAS (G12C). Drug target pathway is ERK MAPK signaling.
The mutations of the cell line are AKT3, APC, CARD11, FANCL, FLT1, FOXL2, GATA3, IRF4, NF1, PBRM1, PIK3CA, POLE, STAG2, TP53.
Drug Sensitivity: Sensitive

Example 1:

Drug and cell line mutations: 
The drug is LY2109761. The drug SMILES structure is C1CC2=C(C(=NN2C1)C3=CC=CC=N3)C4=C5C=CC(=CC5=NC=C4)OCCN6CCOCC6. Drug target is TGFB1. Drug target pathway is Other.
The mutations of the cell line are ATR, ATRX, BRAF, BRCA2, CDH1, EZH2, MYCN, NOTCH2, PBRM1, PTPN11, ROS1, SF3B1, SGK1, SOX9, TBX3, TSC2.
Drug Sensitivity: Sen

/data/yingfei/models/llm/medAlpaca/medalpaca_env/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


KeyboardInterrupt: 

In [9]:
import pandas as pd

df_result = pd.read_csv('test_output.csv')
df_result.head()

,cell_id,prompt,answer,output70
0,ACH-000006,Think step by step and decide in a single word...,Sensitive,<</SYS>>
1,ACH-000006,Think step by step and decide in a single word...,Resistant,<<SYS>>Think step by step and decide in a sing...
2,ACH-000006,Think step by step and decide in a single word...,Resistant,<</SYS>>
3,ACH-000006,Think step by step and decide in a single word...,Sensitive,<<SYS>>The sensitivity of the drug on the cell...
4,ACH-000006,Think step by step and decide in a single word...,Sensitive,NaN


In [11]:
# pred_lst = []
# for x in df_result.output70:
#     pre = x.find('Resistant')
#     if pre != -1:
#         pred_lst.append('Resistant')
#     else:
#         pred_lst.append('Sensitive')

In [ ]:
pred_lst[:5]

In [ ]:
corr = [1 for idx, x in enumerate(df_result.answer) if x == pred_lst[idx]]

In [ ]:
sum(corr)/len(df_result)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(df_result.answer.tolist(), pred_lst, labels=["Resistant", "Sensitive"])

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Resistant", "Sensitive"])

In [ ]:
disp.plot()